### Trade Demo

#### Goals:
- Login to the Canada domain.
- Select the dataset.
- Cacluate the sum of total of good imported and exported to Egypt.
- Request for the sum
- Download the results

### Step 1: Login into the Canada domain

In [1]:
# As a Data Scientist we want to perform some analysis on the trade dataset available on the Canada domain.
# Our goal is to calculate the sum total of the goods imported (imports + re-imports) 
# and exported (exports + re-exports) to Egypt.

import syft as sy

# Let's login into the canada domain
canada_domain_node = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8081)

Connecting to http://localhost:8081... done! 	 Logging into canada... done!


### Step 2: Select the dataset

In [3]:
# Awesome !!! We're logged into the domain node
# Let's quickly list all the datasets available on the Canada domain

canada_domain_node.datasets

,description,data,id,tags,name
0,A collection of reports from Canada's statisti...,"[{'name': 'feb2020', 'id': '6e706f2a-b7a0-4b2d...",a17630d2-6382-40e6-8c5e-37f996fafdd2,[],Canada Trade Data - First 10000 rows


In [4]:
canada_domain_node.datasets[0]

,name,data,id,tags,description
0,Canada Trade Data - First 10000 rows,"[{'name': 'feb2020', 'id': '6e706f2a-b7a0-4b2d...",a17630d2-6382-40e6-8c5e-37f996fafdd2,[],A collection of reports from Canada's statisti...


In [5]:
feb2020 = canada_domain_node.datasets[0]['feb2020']
feb2020.tags

['#feb2020']

In [6]:
# # Let's try to get the whole dataset itself.
feb2020.request(reason="Access whole dataset")

### STOP: Return to Data Owner-Canada.ipynb - STEP 6!!

# Step 3: Try to download the whole dataset

In [8]:
# Let's see if our request was accepted
feb2020.get()

[2021-08-06T12:40:03.769115+0100][CRITICAL][logger]][29612] You do not have permission to .get() Object with ID: <UID: 6e706f2ab7a04b2d8cbae7429ef4e41b>Please submit a request.


AuthorizationException: You do not have permission to .get() Object with ID: <UID: 6e706f2ab7a04b2d8cbae7429ef4e41b>Please submit a request.

In [9]:
# Seems, like the request to download the whole dataset was denied by the Data Owner.

# Step 4: Do some analysis

In [10]:
# Let's continue with the analysis remotely.
# Let's calculate the value of the goods import from Egypt.

value_of_goods_imported_from_egypt = feb2020[(feb2020['Partner'] == 'Egypt') & (feb2020['Trade Flow'] == 'Imports')]['Trade Value (US$)'].sum()
value_of_goods_reimported_from_egypt = feb2020[(feb2020['Partner'] == 'Egypt') & (feb2020['Trade Flow'] == 'Re-imports')]['Trade Value (US$)'].sum()

In [11]:
value_of_goods_imported_from_egypt.request(reason="Goods value imported from Egypt")

In [12]:
value_of_goods_reimported_from_egypt.request(reason="Goods value imported from Egypt")

### STOP Return to Data Owner - Canada.ipynb step 7

# Step 8: Download Results

In [13]:
# wait for data owner to accept/deny request

In [14]:
imported_goods_value = value_of_goods_imported_from_egypt.get(delete_obj=False)
reimported_goods_value = value_of_goods_reimported_from_egypt.get(delete_obj=False)
print(f"Total trade value($) of the goods Imported from Egypt to Canada: ${imported_goods_value + reimported_goods_value}")

Total trade value($) of the goods Imported from Egypt to Canada: $4332428


*Great !!! We successfully, finished our analysis.*

# CONGRATULATIONS!!!

You've finished the demo!